# Analysis code for the paper "B1+ shimming in the cervical spinal cord at 7T"

## Data

The data can be downloaded at https://openneuro.org/datasets/ds004906

The structure of the input dataset is as follows (JSON sidecars are not listed for clarity):
~~~
ds004906
├── CHANGES
├── README
├── dataset_description.json
├── participants.json
├── participants.tsv
├── sub-01
│   ├── anat
│   │   ├── sub-01_acq-CP_T1w.nii.gz
│   │   ├── sub-01_acq-CP_T2starw.nii.gz
│   │   ├── sub-01_acq-CoV_T1w.nii.gz
│   │   ├── sub-01_acq-CoV_T2starw.nii.gz
│   │   ├── sub-01_acq-SAReff_T2starw.nii.gz
│   │   ├── sub-01_acq-patient_T2starw.nii.gz
│   │   ├── sub-01_acq-phase_T2starw.nii.gz
│   │   ├── sub-01_acq-target_T2starw.nii.gz
│   │   ├── sub-01_acq-volume_T2starw.nii.gz
│   └── fmap
│       ├── sub-01_acq-anatCP_TB1TFL.nii.gz
│       ├── sub-01_acq-anatCoV_TB1TFL.nii.gz
│       ├── sub-01_acq-anatSAReff_TB1TFL.nii.gz
│       ├── sub-01_acq-anatpatient_TB1TFL.nii.gz
│       ├── sub-01_acq-anatphase_TB1TFL.nii.gz
│       ├── sub-01_acq-anattarget_TB1TFL.nii.gz
│       ├── sub-01_acq-anatvolume_TB1TFL.nii.gz
│       ├── sub-01_acq-fampCP_TB1TFL.nii.gz
│       ├── sub-01_acq-fampCoV_TB1TFL.nii.gz
│       ├── sub-01_acq-fampSAReff_TB1TFL.nii.gz
│       ├── sub-01_acq-famppatient_TB1TFL.nii.gz
│       ├── sub-01_acq-fampphase_TB1TFL.nii.gz
│       ├── sub-01_acq-famptarget_TB1TFL.nii.gz
│       └── sub-01_acq-fampvolume_TB1TFL.nii.gz
├── sub-02
├── sub-03
├── sub-04
└── sub-05
~~~


## Overview of processing pipeline

For each subject:

- Segment the spinal cord on GRE scan
- Label vertebral levels on GRE scan using existing manual disc labels
- Extract the signal intensity on the GRE scan within the spinal cord
- Register each B1 map (CP, CoV, etc.) to the GRE scan
- Apply the computed warping field to bring the segmentation and vertebral levels to the B1 map
- Convert the B1 map to nT/V units
- Extract the B1 map value within the spinal cord
- Segment the spinal cord on the CoV MPRAGE scan
- Label the vertebral levels using automatic labeling
- Register the CP mode MPRAGE to the CoV scan
- Warp segmentation and labeling to the CP mode scan
- Visualize the CP mode and CoV mode MPRAGE scan, calculate coefficient of variation within the cord for both, plot signal intensity

>Slow processes are indicated with the emoji ⏳

In [ ]:
# Necessary imports

import os
import re
import json
import subprocess
import glob
import matplotlib.pyplot as plt
import numpy as np
import nibabel as nib
import pandas as pd
from scipy.interpolate import interp1d
from scipy.ndimage import uniform_filter1d
from scipy.stats import f_oneway
from statsmodels.stats.multicomp import pairwise_tukeyhsd

In [ ]:
# Checking where we are

!pwd

In [ ]:
# Install packages on system
# ℹ️ No need to run this cell if you run this notebook locally and already have these dependencies installed.

!sudo apt-get update
!sudo apt-get install git-annex

# Install Python libaries
!wget -O requirements.txt https://raw.githubusercontent.com/shimming-toolbox/rf-shimming-7t/main/requirements.txt
!pip install -r requirements.txt

In [ ]:
# Install SCT ⏳
# ℹ️ No need to run this cell if you run this notebook locally and already have SCT installed.

!git clone --depth 1 https://github.com/spinalcordtoolbox/spinalcordtoolbox.git
!yes | spinalcordtoolbox/install_sct
# Add SCT's binaries to environment PATH
os.environ['PATH'] += f":/content/spinalcordtoolbox/bin"

In [ ]:
# Download data and define path variables

!datalad install https://github.com/OpenNeuroDatasets/ds004906.git
os.chdir("ds004906")
!datalad get . # uncomment for production
# !datalad get sub-01/  # debugging
# Get derivatives containing manual labels
!datalad get derivatives

In [ ]:
# Define useful variables

path_data = os.getcwd()
print(f"path_data: {path_data}")
path_qc = os.path.join(path_data, "qc")
shim_modes = ["CP", "patient", "volume", "phase", "CoV", "target", "SAReff"]
shim_modes_MPRAGE = ["CP", "CoV"]  # TODO: change variable name PEP8
# shim_modes = ["CP", "CoV"]  # debugging
print(f"shim_modes: {shim_modes}")
subjects = sorted(glob.glob("sub-*"))
print(f"subjects: {subjects}")

# Create output folder
path_results = os.path.join(path_data, 'derivatives', 'results')
os.makedirs(path_results, exist_ok=True)

## Process anat/T2starw (GRE)

In [ ]:
# Run segmentation on GRE scan
# ℹ️ The "CoV reduction" RF shimming scenario was chosen as the segmentation baseline due to the more 
# homogeneous signal intensity in the I-->S direction, which results in a better segmentation peformance
# in the C7-T2 region

for subject in subjects:
    os.chdir(os.path.join(path_data, subject, "anat"))
    # Use another syntax for sub-04. See: https://github.com/shimming-toolbox/rf-shimming-7t/issues/31
    if subject == 'sub-04':
        !sct_deepseg_sc -i {subject}_acq-CoV_T2starw.nii.gz -c t2 -qc {path_qc}
    else:
        !sct_deepseg_sc -i {subject}_acq-CoV_T2starw.nii.gz -c t2s -qc {path_qc}

In [ ]:
# Crop GRE scan for faster processing and better registration results

for subject in subjects:
    os.chdir(os.path.join(path_data, subject, "anat"))
    !sct_crop_image -i {subject}_acq-CoV_T2starw.nii.gz -m {subject}_acq-CoV_T2starw_seg.nii.gz -dilate 20x20x0 -o {subject}_acq-CoV_T2starw_crop.nii.gz
    !sct_crop_image -i {subject}_acq-CoV_T2starw_seg.nii.gz -m {subject}_acq-CoV_T2starw_seg.nii.gz -dilate 20x20x0 -o {subject}_acq-CoV_T2starw_crop_seg.nii.gz

In [ ]:
# Label vertebrae on GRE scan

# Given the low resolution of the GRE scan, the automatic detection of C2-C3 disc is unreliable. Therefore we need to use the manual disc labels that are part of the dataset.
for subject in subjects:
    os.chdir(os.path.join(path_data, subject, "anat"))
    fname_label_discs = os.path.join(path_data, "derivatives", "labels", subject, "anat", f"{subject}_acq-CoV_T2starw_label-discs_dseg.nii.gz")
    !sct_label_utils -i {subject}_acq-CoV_T2starw_crop_seg.nii.gz -disc {fname_label_discs} -o {subject}_acq-CoV_T2starw_crop_seg_labeled.nii.gz
    # Generate QC report to assess labeled segmentation
    !sct_qc -i {subject}_acq-CoV_T2starw_crop.nii.gz -s {subject}_acq-CoV_T2starw_crop_seg_labeled.nii.gz -p sct_label_vertebrae -qc {path_qc} -qc-subject {subject}

In [ ]:
# Register *_T2starw to CoV_T2starw

# Commenting for now, due to https://github.com/shimming-toolbox/rf-shimming-7t/issues/35
for subject in subjects:
    os.chdir(os.path.join(path_data, subject, "anat"))
    for shim_mode in shim_modes:
        # Don't do it for CoV_T2starw
        if shim_mode != 'CoV':
            !sct_register_multimodal -i {subject}_acq-{shim_mode}_T2starw.nii.gz -d {subject}_acq-CoV_T2starw_crop.nii.gz -dseg {subject}_acq-CoV_T2starw_crop_seg.nii.gz -param step=1,type=im,algo=slicereg,metric=CC -qc {path_qc}

In [ ]:
# Create CSF mask by dilating the spinal cord segmentation

for subject in subjects:
    os.chdir(os.path.join(path_data, subject, "anat"))
    !sct_maths -i {subject}_acq-CoV_T2starw_crop_seg.nii.gz -dilate 3 -shape disk -dim 2 -o {subject}_acq-CoV_T2starw_crop_seg_dilate.nii.gz
    !sct_maths -i {subject}_acq-CoV_T2starw_crop_seg_dilate.nii.gz -sub {subject}_acq-CoV_T2starw_crop_seg.nii.gz -o {subject}_acq-CoV_T2starw_crop_CSFseg.nii.gz
    # Generate QC report to assess CSF mask
    !sct_qc -i {subject}_acq-CoV_T2starw_crop.nii.gz -s {subject}_acq-CoV_T2starw_crop_CSFseg.nii.gz -p sct_deepseg_sc -qc {path_qc} -qc-subject {subject}

### Verify QC report (GRE segmentation)

Open the quality control (QC) report located under `ds004906/qc/index.html`. Make sure the spinal cord segmentations are correct before resuming the analysis.

>If you run this notebook on Google Colab, you can skip as the QC report cannot easily be viewed from Google Colab. If you *really* want to see the QC report, you can create a cell where you zip the 'qc/' folder, then you can download it on your local station and open the index.html file. 


In [ ]:
# Extract the signal intensity on the GRE scan within the spinal cord between levels C3 and T2 (included), which correspond to the region where RF shimming was prescribed

for subject in subjects:
    os.chdir(os.path.join(path_data, subject, "anat"))
    for shim_method in ['CoV', 'CP']:  # TODO: shim_modes:
        # Shim methods are registered to the CoV T2starw scan, so we need to use the added suffix to identify them
        if shim_method == 'CoV':
            file_suffix = 'crop'
        else:
            file_suffix = 'reg'
        fname_result_sc = os.path.join(path_results, f"{subject}_acq-{shim_method}_T2starw_label-SC.csv")
        !sct_extract_metric -i {subject}_acq-{shim_method}_T2starw_{file_suffix}.nii.gz -f {subject}_acq-CoV_T2starw_crop_seg.nii.gz -method wa -vert 3:9 -vertfile {subject}_acq-CoV_T2starw_crop_seg_labeled.nii.gz -perslice 1 -o {fname_result_sc}
        fname_result_csf = os.path.join(path_results, f"{subject}_acq-{shim_method}_T2starw_label-CSF.csv")
        !sct_extract_metric -i {subject}_acq-{shim_method}_T2starw_{file_suffix}.nii.gz -f {subject}_acq-CoV_T2starw_crop_CSFseg.nii.gz -method wa -vert 3:9 -vertfile {subject}_acq-CoV_T2starw_crop_seg_labeled.nii.gz -perslice 1 -o {fname_result_csf}

In [ ]:
# Make figure of SC/CSF signal ratio from T2starw scan

# Go back to root data folder
os.chdir(path_data)

def smooth_data(data, window_size=20):
    """ Apply a simple moving average to smooth the data. """
    return uniform_filter1d(data, size=window_size, mode='nearest')

# Fixed grid for x-axis
x_grid = np.linspace(0, 1, 100)

# z-slices corresponding to levels C1 to T2 on the PAM50 template. These will be used to scale the x-label of each subject.
original_vector = np.array([984, 938, 907, 870, 833, 800, 769, 735, 692, 646])

# Normalize the PAM50 z-slice numbers to the 1-0 range (to show inferior-superior instead of superior-inferior)
min_val = original_vector.min()
max_val = original_vector.max()
normalized_vector = 1 - ((original_vector - min_val) / (max_val - min_val))

# Use this normalized vector as x-ticks
custom_xticks = normalized_vector

# Number of subjects determines the number of rows in the subplot
n_rows = len(subjects)

# Create a figure with multiple subplots
fig, axes = plt.subplots(n_rows, 1, figsize=(10, 6 * n_rows))
font_size = 18

# Check if axes is an array or a single object
if n_rows == 1:
    axes = [axes]

# Iterate over each subject and create a subplot
for i, subject in enumerate(subjects):
    ax = axes[i]

    for shim_method in ['CoV', 'CP']:  # TODO: shim_modes:
        # Initialize list to collect data for this shim method
        method_data = []

        file_csv = os.path.join(path_results, f"{subject}_acq-{shim_method}_T2starw_label-SC.csv")
        df = pd.read_csv(file_csv)
        wa_data = df['WA()']

        # Normalize the x-axis to a 0-1 scale for each subject
        x_subject = np.linspace(0, 1, len(wa_data))

        # Interpolate to the fixed grid
        interp_func = interp1d(x_subject, wa_data, kind='linear', bounds_error=False, fill_value='extrapolate')
        resampled_data = interp_func(x_grid)

        # Apply smoothing
        smoothed_data = smooth_data(resampled_data)

        method_data.append(smoothed_data)

        # If there's data for this shim method, plot it
        if method_data:
            # Plotting each file's data separately
            for resampled_data in method_data:
                ax.plot(x_grid, resampled_data, label=f"{shim_method}")

    # Set custom x-ticks
    ax.set_xticks(custom_xticks)
    ax.set_xticklabels([''] * len(original_vector))

    ax.set_title(f'{subject}', fontsize=font_size)
    ax.set_ylabel('Cord/CSF T2starw signal ratio', fontsize=font_size)
    ax.tick_params(axis='y', which='major', labelsize=font_size-4)

    # Add legend only to the first subplot
    if i == 0:
        ax.legend(fontsize=font_size)

    ax.grid(True)

# Adjust the layout so labels and titles do not overlap
plt.tight_layout()
plt.show()

## Process fmap/TFL (flip angle maps)

In [ ]:
# Register TFL flip angle maps to the GRE scan ⏳

for subject in subjects:
    os.chdir(os.path.join(path_data, subject, "fmap"))
    for shim_mode in shim_modes:
        !sct_register_multimodal -i {subject}_acq-anat{shim_mode}_TB1TFL.nii.gz -d ../anat/{subject}_acq-CoV_T2starw_crop.nii.gz -dseg ../anat/{subject}_acq-CoV_T2starw_crop_seg.nii.gz -param step=1,type=im,algo=slicereg,metric=CC -qc {path_qc}

### Verify QC report (B1maps to GRE registration)

Open the QC report located under `ds004906/qc/index.html`. Make sure the registration are correct before resuming the analysis.

In [ ]:
# Warping spinal cord segmentation and vertebral level to each flip angle map

for subject in subjects:
    os.chdir(os.path.join(path_data, subject, "fmap"))
    for shim_mode in shim_modes:
        # Use linear interpolation to preserve partial volume information (needed when extracting values along the cord)
        !sct_apply_transfo -i ../anat/{subject}_acq-CoV_T2starw_crop_seg.nii.gz -d {subject}_acq-anat{shim_mode}_TB1TFL.nii.gz -w warp_{subject}_acq-CoV_T2starw_crop2{subject}_acq-anat{shim_mode}_TB1TFL.nii.gz -x linear -o {subject}_acq-anat{shim_mode}_TB1TFL_seg.nii.gz
        # Use nearest neighbour (nn) interpolation because we are dealing with non-binary discrete labels
        !sct_apply_transfo -i ../anat/{subject}_acq-CoV_T2starw_crop_seg_labeled.nii.gz -d {subject}_acq-anat{shim_mode}_TB1TFL.nii.gz -w warp_{subject}_acq-CoV_T2starw_crop2{subject}_acq-anat{shim_mode}_TB1TFL.nii.gz -x nn -o {subject}_acq-anat{shim_mode}_TB1TFL_seg_labeled.nii.gz

In [ ]:
# Convert the flip angle maps to B1+ efficiency maps [nT/V] (inspired by code from Kyle Gilbert)
# The approach consists in calculating the B1+ efficiency using a 1ms, pi-pulse at the acquisition voltage,
# then scale the efficiency by the ratio of the measured flip angle to the requested flip angle in the pulse sequence.

GAMMA = 2.675e8;  # [rad / (s T)]
requested_fa = 90  # saturation flip angle -- hard-coded in sequence

for subject in subjects:
    os.chdir(os.path.join(path_data, subject, "fmap"))
    for shim_mode in shim_modes:
        # Fetch the reference voltage from the JSON sidecar to the TFL B1map sequence
        with open(f"{subject}_acq-famp{shim_mode}_TB1TFL.json", "r") as f:
            metadata = json.load(f)
            ref_voltage = metadata.get("TxRefAmp", "N/A")
            print(f"ref_voltage [V]: {ref_voltage} ({subject}_acq-famp{shim_mode}_TB1TFL)")

        # Open flip angle map with nibabel
        nii = nib.load(f"{subject}_acq-famp{shim_mode}_TB1TFL.nii.gz")
        acquired_fa = nii.get_fdata()

        # Siemens maps are in units of flip angle * 10 (in degrees)
        acquired_fa = acquired_fa / 10

        # Account for the power loss between the coil and the socket. That number was given by Siemens.
        voltage_at_socket = ref_voltage * 10 ** -0.095

        # Compute B1 map in [T/V]
        b1_map = (acquired_fa / requested_fa) * (np.pi / (GAMMA * 1e-3 * voltage_at_socket))

        # Convert to [nT/V]
        b1_map = b1_map * 1e9

        # Save as NIfTI file
        nii_b1 = nib.Nifti1Image(b1_map, nii.affine, nii.header)
        nib.save(nii_b1, f"{subject}_acq-{shim_mode}_TB1map.nii.gz")

In [ ]:
# Extract B1+ value along the spinal cord between levels C3 and T2 (included)

for subject in subjects:
    os.chdir(os.path.join(path_data, subject, "fmap"))
    for shim_mode in shim_modes:
        !sct_extract_metric -i {subject}_acq-{shim_mode}_TB1map.nii.gz -f {subject}_acq-anat{shim_mode}_TB1TFL_seg.nii.gz -method wa -vert 3:9 -vertfile {subject}_acq-anat{shim_mode}_TB1TFL_seg_labeled.nii.gz -perslice 1 -o TB1map_{shim_mode}.csv

In [ ]:
# Make figure of B1+ values along the spinal cord across shim methods

# Go back to root data folder
os.chdir(os.path.join(path_data))

def smooth_data(data, window_size=20):
    """ Apply a simple moving average to smooth the data. """
    return uniform_filter1d(data, size=window_size, mode='nearest')

# Fixed grid for x-axis
x_grid = np.linspace(0, 1, 100)

# z-slices corresponding to levels C1 to T2 on the PAM50 template. These will be used to scale the x-label of each subject.
original_vector = np.array([984, 938, 907, 870, 833, 800, 769, 735, 692, 646])

# Normalize the PAM50 z-slice numbers to the 1-0 range (to show inferior-superior instead of superior-inferior)
min_val = original_vector.min()
max_val = original_vector.max()
normalized_vector = 1 - ((original_vector - min_val) / (max_val - min_val))

# Use this normalized vector as x-ticks
custom_xticks = normalized_vector

# Number of subjects determines the number of rows in the subplot
n_rows = len(subjects)

# Create a figure with multiple subplots
fig, axes = plt.subplots(n_rows, 1, figsize=(10, 6 * n_rows))
font_size = 18

# Check if axes is an array or a single object
if n_rows == 1:
    axes = [axes]

# Iterate over each subject and create a subplot
for i, subject in enumerate(subjects):
    ax = axes[i]
    
    os.chdir(os.path.join(path_data, subject, "fmap"))

    for shim_method in shim_modes:
        # Initialize list to collect data for this shim method
        method_data = []

        for file_path in glob.glob(f"TB1map_*{shim_method}*.csv"):
            df = pd.read_csv(file_path)
            wa_data = df['WA()']

            # Normalize the x-axis to a 0-1 scale for each subject
            x_subject = np.linspace(0, 1, len(wa_data))

            # Interpolate to the fixed grid
            interp_func = interp1d(x_subject, wa_data, kind='linear', bounds_error=False, fill_value='extrapolate')
            resampled_data = interp_func(x_grid)

            # Apply smoothing
            smoothed_data = smooth_data(resampled_data)

            method_data.append(smoothed_data)

        # If there's data for this shim method, plot it
        if method_data:
            # Plotting each file's data separately
            for resampled_data in method_data:
                ax.plot(x_grid, resampled_data, label=f"{shim_method}")

    # Set custom x-ticks
    ax.set_xticks(custom_xticks)
    ax.set_xticklabels([''] * len(original_vector))

    ax.set_title(f'{subject}', fontsize=font_size)
    ax.set_ylabel('B1+ efficiency [nT/V]', fontsize=font_size)
    ax.tick_params(axis='y', which='major', labelsize=font_size-4)

    # Add legend only to the first subplot
    if i == 0:
        ax.legend(fontsize=font_size)

    ax.grid(True)

# Adjust the layout so labels and titles do not overlap
plt.tight_layout()
plt.show()

In [ ]:
# Create tables and perform statistics

# Go back to root data folder
os.chdir(path_data)

# Data storage
data_summary = []

# Compute mean and SD of B1+ for each subject and each shim mode
for subject in subjects:
    for shim_mode in shim_modes:
        all_data = []
        for file_path in glob.glob(os.path.join(path_data, subject, "fmap", f"TB1map_*{shim_mode}*.csv")):
            df = pd.read_csv(file_path)
            # Drop rows where VertLevel is 1 or 2 (because RF shimming was not performed at these levels)
            df = df[df['VertLevel'].isin(['1', '2']) == False]
            wa_data = df['WA()']
            all_data.extend(wa_data)

        if all_data:
            mean_data = np.mean(all_data)
            sd_data = np.std(all_data)
            data_summary.append([subject, shim_mode, mean_data, sd_data])

# Convert to DataFrame and save to CSV
df_summary = pd.DataFrame(data_summary, columns=['Subject', 'Shim_Mode', 'Average', 'Standard_Deviation'])
df_summary.to_csv(os.path.join(path_results, 'subject_shim_mode_summary.csv'), index=False)

# Compute statistics across subjects
grouped_means = df_summary.groupby('Shim_Mode').agg({'Average': ['mean', 'std'], 'Standard_Deviation': ['mean', 'std']})

# Format mean ± standard deviation
grouped_means['Average_formatted'] = grouped_means['Average']['mean'].map("{:.2f}".format) + " ± " + grouped_means['Average']['std'].map("{:.2f}".format)
grouped_means['Standard_Deviation_formatted'] = grouped_means['Standard_Deviation']['mean'].map("{:.2f}".format) + " ± " + grouped_means['Standard_Deviation']['std'].map("{:.2f}".format)

# Drop multi-level index and only keep formatted columns
grouped_means = grouped_means.drop(columns=['Average', 'Standard_Deviation'])
grouped_means.columns = ['Average', 'Standard_Deviation']  # Rename columns for clarity
grouped_means.reset_index().to_csv(os.path.join(path_results, 'average_across_subjects.csv'), index=False)

# Perform ANOVA and Posthoc Tests
anova_result = f_oneway(*[group["Average"].values for name, group in df_summary.groupby("Shim_Mode")])
print("ANOVA Result:", anova_result)

if anova_result.pvalue < 0.05:
    posthoc_result = pairwise_tukeyhsd(df_summary['Average'], df_summary['Shim_Mode'])
    print("Posthoc Tukey HSD Result:\n", posthoc_result)


In [ ]:
# Visualize RF maps to generate Figure 2

# TODO